In [1]:
import pandas as pd
import numpy as np
import time
import re
import json
import urllib.request

#获取汇率
def get_rate():
    
    t = time.time()
    # timestamp must be ms
    timestamp = (int(round(t*1000)))
    url = "http://www.chinamoney.com.cn/r/cms/www/chinamoney/data/fx/ccpr.json?t="+str(timestamp)
    req = urllib.request.Request(url)
    f = urllib.request.urlopen(req)
    html = f.read().decode("utf-8")
    rjson = json.loads(html)
    

    for r in rjson['records']:
        if r['vrtEName'] == 'USD/CNY':
            return float(r['price'])

#价格清洗        
def convert_currency(value):
    """
    转换字符串数字为float类型
     - 移除 ￥ ,
     - 转化为float类型
    """
    new_value = value.replace(',', '').replace('**',"").replace('*',"").replace('$', '')
   # print(new_value)
    return  np.float(new_value)

#数据采集
def get_data(sym):
    url="https://coinmarketcap.com/zh/currencies/"+sym+"/#markets"
    data =pd.read_html(url)[0]
   
    data['supply']=data['交易量（24小时）'].apply(convert_currency)

    data['price']=data['价格'].apply(convert_currency)

    data['exchange']=data['来源']
    data['sym']=sym

    df = pd.DataFrame(data)
    x=[0,1,2,3,4,5,6,7,8]
    df.drop(df.columns[x], axis=1, inplace=True) 
    return df

#数据采集

def merge_data():
    coins =  {
              "0x" : ['Poloniex','Huobi','HitBTC','CoinTiger','Binance','Gate.io','OKEx','Ethfinex','Upbit','Bittrex','Bithumb','Coinbase Pro','BitMart','DigiFinex'],
              "kyber-network" : ['Huobi','OKEx','Liqui','Gate.io','Binance','DragonEX','Bithumb','CPDAX'] ,
              "loopring" : ['OKEx','IDAX','Binance','Bithumb','Gate.io','Bittrex','Upbit','DragonEX'] ,
              "loopring-neo" : ['Gate.io'] 
             }
     #coin = {k: f(v) for k, v in coins.items()}
       
    data=[]
    for key,values in  coins.items():
        df=get_data(key)
        data.append(df.loc[df['exchange'].isin(values)])
    
    df=pd.concat(data)
    #frames = [df1, df2, df3]
    #result = pd.concat(frames)
    return df

df=merge_data() 
df

 

dict_mapping = {
                'price':'mean',
                'supply':'sum'}
dd=df.groupby(['exchange','sym']).agg(dict_mapping)

r=get_rate()
print('当前汇率:',r)
def rate(val): 
    return r*val

dd['cny_supply']=dd['supply'].apply(rate).apply(lambda x:   x/10000).apply(lambda x: '%.2f' % x)

dd['cny_price']=dd['price'].apply(rate).apply(lambda x: '%.2f' % x)

dd.sort_values(by=['supply'])

dd.T
dd.to_csv('ok.csv')
dd 

当前汇率: 6.9453


price     supply cny_supply cny_price
exchange     sym                                                    
Binance      0x             0.350104  2039206.0    1416.29      2.43
             kyber-network  0.161873   341642.0     237.28      1.12
             loopring       0.042222   117525.0      81.62      0.29
BitMart      0x             0.311443  3894545.0    2704.88      2.16
Bithumb      0x             0.354103   145957.0     101.37      2.46
             kyber-network  0.165071   235581.0     163.62      1.15
             loopring       0.315054    65563.0      45.54      2.19
Bittrex      0x             0.352400   113385.0      78.75      2.45
             loopring       0.041637    13195.0       9.16      0.29
CPDAX        kyber-network  0.161773   248473.0     172.57      1.12
CoinTiger    0x             0.351235   199256.0     138.39      2.44
Coinbase Pro 0x             0.353228  3237849.0    2248.78      2.45
DigiFinex    0x             0.350204   331182.0     230.02      2.43
DragonEX     kyber-network  0.149537   258223.0     179.34      1.04
             loopring       0.045319    85066.0      59.08      0.31
Ethfinex     0x             0.355228   580222.0     402.98      2.47
Gate.io      0x             0.350604   143624.0      99.75      2.44
             kyber-network  0.164946     4895.0       3.40      1.15
             loopring       0.041762    80162.0      55.67      0.29
             loopring-neo   0.071477   627316.0     435.69      0.50
HitBTC       0x             0.356339   172074.0     119.51      2.47
Huobi        0x             0.351222    89877.0      62.42      2.44
             kyber-network  0.161954    45506.0      31.61      1.12
IDAX         loopring       0.049938        0.0       0.00      0.35
Liqui        kyber-network  0.162446   129531.0      89.96      1.13
OKEx         0x             0.350979   361570.0     251.12      2.44
             kyber-network  0.162382    41136.0      28.57      1.13
             loopring       0.042080    98639.0      68.51      0.29
Poloniex     0x             0.350015   174804.0     121.41      2.43
Upbit        0x             0.353969   105852.0      73.52      2.46
             loopring       0.041864    12279.0       8.53      0.29